In [39]:
import numpy as np
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F


# Define the GCN Model
class GCN(torch.nn.Module):
    """
    class GCNConv(
        in_channels: int,
        out_channels: int,
        improved: bool = False,
        cached: bool = False,
        add_self_loops: bool | None = None,
        normalize: bool = True,
        bias: bool = True,
        **kwargs: Any
    )
    """

    def __init__(self, num_features, num_outputs):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_outputs)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        # Pool node features within each graph to a single vector
        # x = global_mean_pool(x, batch)
        # return F.log_softmax(x, dim=1)
        return x


# Generate some example data
def create_random_graph_data(
    num_node_features, num_classes=None, min_nodes=5, max_nodes=20
):
    num_nodes = np.random.randint(
        min_nodes, max_nodes + 1
    )  # Randomly choose the number of nodes
    edge_prob = 0.2  # Probability of edge creation between any two nodes
    edge_index = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j and np.random.rand() < edge_prob:
                edge_index.append([i, j])
    edge_index = (
        torch.tensor(edge_index).t().contiguous()
    )  # Transpose and make contiguous
    x = torch.randn(num_nodes, num_node_features)  # Node features
    # y = torch.randint(0, num_classes, (1,)).item()  # Graph-level label
    # return Data(x=x, edge_index=edge_index, y=y)
    return Data(x=x, edge_index=edge_index)


num_graphs = 4

# Use the new function to create a dataset with more varied graph sizes
graphs = [create_random_graph_data(3, 2) for _ in range(num_graphs)]

# DataLoader for batching graphs
loader = DataLoader(graphs, batch_size=4, shuffle=False)

# Initialize the model
model = GCN(num_features=3, num_outputs=8)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(10):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)

        print(out.shape)

        loss = F.nll_loss(out, batch.y)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

torch.Size([62, 8])


/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


TypeError: nll_loss_nd(): argument 'target' (position 2) must be Tensor, not NoneType

In [40]:
graphs

[Data(x=[13, 3], edge_index=[2, 29]),
 Data(x=[14, 3], edge_index=[2, 37]),
 Data(x=[16, 3], edge_index=[2, 43]),
 Data(x=[19, 3], edge_index=[2, 68])]

In [41]:
graphs[-1].edge_index

tensor([[ 0,  0,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  3,  3,  4,  4,  4,  4,
          4,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  9,  9,
         10, 10, 10, 11, 11, 11, 11, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14,
         14, 15, 16, 16, 16, 16, 16, 17, 17, 17, 18, 18, 18, 18],
        [ 1,  6,  2,  4,  5,  8, 14, 17, 18,  0,  5, 10,  6, 14,  0,  5,  7,  8,
         10,  1,  2,  3,  4, 13, 16,  0,  1, 11, 17,  2,  4,  5,  8, 17,  0, 17,
          5,  8, 18,  3,  4,  5, 17,  5, 18,  4, 11, 12, 16, 17,  2,  3,  9, 12,
         16, 12,  2,  7,  8, 10, 15,  4, 10, 12,  5,  6, 15, 17]])

In [42]:
batch

DataBatch(x=[62, 3], edge_index=[2, 177], batch=[62], ptr=[5])

In [43]:
batch.x

tensor([[-0.1690, -1.0338, -1.6202],
        [-0.2585, -0.7059, -0.1570],
        [ 1.2097,  0.3235, -0.2836],
        [ 0.6167, -0.2658,  0.5984],
        [ 1.6300, -1.6904, -0.2952],
        [ 1.4837, -0.8730,  0.3657],
        [-0.3554,  0.6896, -0.7299],
        [-0.8573, -1.0838, -0.4775],
        [-0.0835, -0.0748,  0.1922],
        [ 0.2450,  0.0071, -0.4362],
        [-0.7499,  0.6503, -0.0319],
        [-0.6749,  2.5527, -1.4721],
        [ 0.1698, -0.6225,  1.0252],
        [ 1.0068, -0.0274, -1.0398],
        [ 0.3309, -0.7549, -0.9624],
        [ 0.7005,  1.1697,  0.0126],
        [-0.5175,  1.3866, -1.8906],
        [-1.4835,  1.0586, -1.2979],
        [ 0.8968,  1.6175, -0.6017],
        [ 0.1674,  0.7094, -0.6425],
        [ 1.0856,  0.6375,  2.6912],
        [ 2.5122, -1.5671, -0.0251],
        [-1.0388,  0.5314, -1.0635],
        [-1.6514,  0.9155, -1.0362],
        [-0.6094, -0.2221, -0.7273],
        [-0.8373, -1.0480,  0.7659],
        [ 0.9466,  2.3291, -0.5461],
 

In [44]:
batch.edge_index

tensor([[ 0,  0,  1,  2,  3,  3,  3,  4,  4,  5,  5,  6,  6,  6,  6,  7,  7,  8,
          8,  8,  8,  9,  9, 10, 10, 11, 11, 11, 12, 13, 13, 13, 14, 14, 15, 15,
         16, 16, 16, 16, 16, 17, 17, 18, 18, 18, 18, 19, 19, 20, 20, 20, 21, 21,
         21, 21, 23, 23, 23, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 28, 28,
         28, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 35, 35, 36,
         36, 37, 37, 37, 38, 38, 38, 38, 38, 39, 39, 40, 40, 40, 41, 42, 42, 42,
         42, 43, 43, 44, 44, 44, 44, 44, 44, 44, 45, 45, 45, 46, 46, 47, 47, 47,
         47, 47, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 50, 50, 50, 50, 51, 52,
         52, 53, 53, 53, 54, 54, 54, 54, 55, 55, 56, 56, 56, 56, 56, 57, 57, 57,
         57, 57, 58, 59, 59, 59, 59, 59, 60, 60, 60, 61, 61, 61, 61],
        [ 1,  3, 12,  8,  0,  4, 11,  6,  9,  4, 11,  0,  1,  2,  8,  1,  2,  5,
          7, 10, 12,  0,  7,  9, 11,  5,  9, 12,  2, 24, 25, 26, 13, 20, 13, 19,
         13, 14, 15, 20, 24, 16, 24, 14

In [46]:
batch.batch

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])